<a href="https://colab.research.google.com/github/mannmoshe/deep-learning/blob/main/FinalProject/InquiriesClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Deep Learning Final Project<br>
Iquieries for Customer Assistant Classification<br>
By Moshe Mann

Classify inquiries for group treatment.

In [3]:
import pandas as pd
import numpy as np
from itertools import islice
from sklearn.model_selection import train_test_split

filepath = 'https://raw.githubusercontent.com/mannmoshe/deep-learning/main/FinalProject/ca_data.csv'
ca_data = pd.read_csv(filepath, sep=',')

In [4]:
ca_data.head()

,ticketNum,Description,groupTreatment,groupTreatmentCode
0,I-6283070,נושא: RE: 352-0723635 בהוראות התכנית המקוונ...,צוות הדרכה והטמעה אפליקטיבי,3
1,I-5482713,נושא: עדכון תכנית מישיבות היי אלי כבר דיברת...,צוות הדרכה והטמעה אפליקטיבי,3
2,I-6301664,נושא: אפשרות חיפוש בחלונית בחירת ישויות מאוחד...,צוות הדרכה והטמעה אפליקטיבי,3
3,I-5543621,נושא: FW: צרופה לתכנית 553-0674457 ממערכת מבאת...,צוות הדרכה והטמעה אפליקטיבי,3
4,I-6239910,"שלום רב, נושא: השמשת מולטימדיה בחדרי החקירות צ...",צוות טכנאים,2


The ticketNum is shuffeled (Not true data).

In [15]:
counts = ca_data['groupTreatment'].value_counts()
percent = counts / counts.sum()
fmt = '{:.1%}'.format
pd.DataFrame({'counts': counts, 'per': percent.map(fmt)})

,counts,per
7878 מרכז שירות ותמיכה טכנולוגית,4455,65.0%
צוות טכנאים,1397,20.4%
צוות הדרכה והטמעה אפליקטיבי,845,12.3%
צוות תשתיות/סיסטם,142,2.1%
תקשורת ואבטחת מידע,8,0.1%
צוות פיקוח על הבניה,3,0.0%
צוות אינטגרציה,2,0.0%
צוות מרכז תמיכה URD,1,0.0%


In [6]:
ca_data[['groupTreatment', 'groupTreatmentCode']].drop_duplicates().sort_values(by=['groupTreatmentCode'])

,groupTreatment,groupTreatmentCode
26,7878 מרכז שירות ותמיכה טכנולוגית,1
4,צוות טכנאים,2
0,צוות הדרכה והטמעה אפליקטיבי,3
6,צוות תשתיות/סיסטם,4
13,צוות מרכז תמיכה URD,4
14,צוות אינטגרציה,4
22,צוות פיקוח על הבניה,4
1888,תקשורת ואבטחת מידע,4


In [16]:
X = ca_data['Description']
y = ca_data['groupTreatmentCode']

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4, stratify=y)

In [38]:
print(x_train.head(1), y_train.head(1))
print(x_train.shape, y_train.shape)

405    נושא: דוח של נוכחות חברי ועדה משנה ב' מחוז תל ...
Name: Description, dtype: object 405    1
Name: groupTreatmentCode, dtype: int64
(4591,) (4591,)


In [39]:
print(x_test.head(1), y_test.head(1))
print(x_test.shape, y_test.shape)

455    נושא: היי , אודה לעזרתכם בדבר שליחת הודעה מה H...
Name: Description, dtype: object 455    1
Name: groupTreatmentCode, dtype: int64
(2262,) (2262,)


In [ ]:
#add here percent of value_counts

PreProcessing

Get top 10,000 frequent words from train data.

In [20]:
all_text = ' '.join(x_train).split()

In [21]:
counts = dict()
for i in all_text:
    counts[i] = counts.get(i, 0) + 1

sorted_counts = {k: v for k, v in sorted(counts.items(), key=lambda item: item[1], reverse=True)}

In [22]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [23]:
most_frequent_d = take(10000, sorted_counts.items())
most_frequent_df = pd.DataFrame.from_dict(most_frequent_d)
most_frequent_df.columns = ['word', 'frequency']
most_frequent_df.index.name = 'Index'
most_frequent_df.reset_index(inplace=True)

In [25]:
most_frequent_df.head(10)

,Index,word,frequency
0,0,נושא:,4593
1,1,את,1386
2,2,-,1168
3,3,לא,1131
4,4,שלום,1009
5,5,של,968
6,6,נשלח,862
7,7,"בברכה,",836
8,8,תודה,826
9,9,מייל:,757


In [26]:
encode_dict = dict(zip(most_frequent_df.word, most_frequent_df.Index))

In [27]:
x_train_df = x_train.to_frame(name='Description')
x_train_df['description_list'] = x_train_df['Description'].str.split()

In [28]:
def encode(l):
    "Encode only words of top 10,000 frequent words"
    lc = l.copy()
    for i in range(len(l)):
        if l[i] in encode_dict:
            lc[i] = encode_dict[l[i]]
        else:
            lc[i] = '?'
    lc = [e for e in lc if e != '?']
    return lc

In [29]:
def full_encode(l):
    "Encode all words, if word not one of top 10,000 frequent words - encode by question mark"
    lc = l.copy()
    for i in range(len(l)):
        if l[i] in encode_dict:
            lc[i] = encode_dict[l[i]]
        else:
            lc[i] = '?'
    return lc

In [30]:
x_train_df['encoded_desc'] = x_train_df['description_list'].map(encode)
x_train_df['full_encoded_desc'] = x_train_df['description_list'].map(full_encode)

In [31]:
x_train_df.head()

,Description,description_list,encoded_desc,full_encoded_desc
405,נושא: דוח של נוכחות חברי ועדה משנה ב' מחוז תל ...,"[נושא:, דוח, של, נוכחות, חברי, ועדה, משנה, ב',...","[0, 795, 5, 555, 368, 127, 272, 914, 38, 174, ...","[0, 795, 5, 555, 368, 127, 272, 914, 38, 174, ..."
6134,נושא: האאוטלוק במחשב האישי שלי לא נפתח,"[נושא:, האאוטלוק, במחשב, האישי, שלי, לא, נפתח]","[0, 1482, 138, 2408, 55, 3, 309]","[0, 1482, 138, 2408, 55, 3, 309]"
4831,"נושא: FW: התנכלות שכנים מבצע קדש 13 היי, נבקש...","[נושא:, FW:, התנכלות, שכנים, מבצע, קדש, 13, הי...","[0, 28, 7434, 7435, 2068, 7436, 828, 31, 1647,...","[0, 28, 7434, 7435, 2068, 7436, 828, 31, 1647,..."
3206,נושא: חיבור מרחוק לא עובד- מתקבלת שגיאת אנטי ו...,"[נושא:, חיבור, מרחוק, לא, עובד-, מתקבלת, שגיאת...","[0, 65, 33, 3, 4642, 536, 2069, 2825, 7439]","[0, 65, 33, 3, 4642, 536, 2069, 2825, 7439]"
2691,נושא: Case CS0142405 is Awaiting your Response...,"[נושא:, Case, CS0142405, is, Awaiting, your, R...","[0, 78, 967, 456, 638, 556, 639, 6, 10, 12, 13...","[0, 78, 967, 456, 638, 556, 639, 6, 10, 12, 13..."


In [32]:
x_test_df = x_test.to_frame(name='Description')
x_test_df['description_list'] = x_test_df['Description'].str.split()
x_test_df['encoded_desc'] = x_test_df['description_list'].map(encode)
x_test_df['full_encoded_desc'] = x_test_df['description_list'].map(full_encode)

In [33]:
x_test_df.head()

,Description,description_list,encoded_desc,full_encoded_desc
455,"נושא: היי , אודה לעזרתכם בדבר שליחת הודעה מה H...","[נושא:, היי, ,, אודה, לעזרתכם, בדבר, שליחת, הו...","[0, 63, 106, 21, 149, 1602, 1451, 229, 140, 42...","[0, 63, 106, 21, 149, 1602, 1451, 229, 140, 42..."
5739,נושא: Femi Recordings Error Report נשלח מ- P...,"[נושא:, Femi, Recordings, Error, Report, נשלח,...","[0, 189, 173, 164, 165, 6, 10, 64, 11, 9, 64, ...","[0, 189, 173, 164, 165, 6, 10, 64, 11, 9, 64, ..."
1782,"נושא: בקשה לעזרה- מחשב שעובד לאט שלום, אבקש א...","[נושא:, בקשה, לעזרה-, מחשב, שעובד, לאט, שלום,,...","[0, 115, 8933, 114, 3609, 2896, 14, 23, 1, 116...","[0, 115, 8933, 114, 3609, 2896, 14, 23, 1, ?, ..."
217,"נושא: סטיק לקליטה של אינטרנט הי, מאחר שהאינטרנ...","[נושא:, סטיק, לקליטה, של, אינטרנט, הי,, מאחר, ...","[0, 5, 838, 157, 605, 259, 3690, 1629, 430, 30...","[0, ?, ?, 5, 838, 157, 605, ?, 259, 3690, 1629..."
2364,"נושא: FW: תכנית 352-0790477 - בוקר טוב, מבקש ...","[נושא:, FW:, תכנית, 352-0790477, -, בוקר, טוב,...","[0, 28, 18, 2, 29, 54, 75, 4206, 1, 26, 91, 17...","[0, 28, 18, ?, 2, 29, 54, 75, 4206, 1, 26, 91,..."


In [34]:
train_data = x_train_df['encoded_desc'].to_numpy()
train_labels = y_train.to_numpy()

test_data = x_test_df['encoded_desc'].to_numpy()
test_labels = y_test.to_numpy()